## <3-6 단어/문장을 벡터로 변환하기>  

프리트레인을 완료된 언어 모델에서 단어, 문장 수준 임베딩을 추출하는 실습을 해봅시다.  
실습엔 미국 자연어 처리 기업 '허깅페이스(huggingface)'가 만든 트랜스포머 라이브러리를 사용합니다.

### 파인튜닝
---
이 책에서 진행하는 실습은 모두 트랜스포머 계열 언어 모델을 사용합니다. 프리트레인을 마친 언어 모델 위에 작은 모듈을 조금 더 쌓아 task를 수행하는 구조입니다. 프리트레인을 마친 BERT와 그 위의 작은 모듈을 포함한 전체 모델을 문서 분류, 개체명 인식 등 다운스트림 데이터로 업데이트하는 과정을 ***파인튜닝(fine-tuning)*** 이라고 합니다.  


BERT의 출력 결과 가운데 어떤 걸 사용하느냐에 따라 크게 두 가지 방식으로 나눠 볼 수 있습니다.

#### 문장 벡터 활용: 문서 분류 등  

문서 분류를 수행하는 모델을 만든다고 하면 그림 3-65와 같은 모양이 됩니다.

<center><그림 3-65 문서 분류></center>

<p align="center"><img src="https://i.imgur.com/5lpkDEB.png">  

<center>출처 : ratsgo's NLPBOOK</center>

그림에서 노란색 박스가 바로 BERT 모델입니다. '빈칸 맞히기'로 프리트레인을 이미 마쳤습니다.   
BERT는 트랜스포머의 인코더 블록(레이어)을 여러 개 쌓은 구조입니다. 그림에서 확인할 수 있듯이 이 블록의 입력과 출력은 단어 시퀀스(정확히는 입력 토큰에 해당하는 벡터들의 시퀀스)이며, 블록 내에서는 입력 단어(토큰 벡터)를 두 개씩 쌍을 지어 서로의 관계를 모두 고려하는 방식으로 계산됩니다.

문장을 워드피스로 토큰화한 후 앞뒤에 문장 시작과 끝을 알리는 스페셜 토큰 **CLS**와 **SEP**를 각각 추가하고 BERT에 입력합니다. 이후 BERT 모델의 마지막 블록의 출력 가운데 **CLS**에 해당하는 벡터를 추출합니다. 그리고 여기에 간단한 처리를 해서 최종 출력(**pooler_output**)을 만듭니다. 트랜스포머 인코더 블록에서는 모든 단어가 서로 영향을 끼치므로 문장 전체(**이 영화 재미없네요**)의 의미가 **pooler_output** 벡터 하나로 응집된 것으로 볼 수 있습니다.

이렇게 뽑은 **pooler_output** 벡터에 작은 모듈을 하나 추가해 그 출력이 미리 정해 놓은 범주(예를 들어 **긍정, 중립, 부정**)가 될 확률이 되도록 합니다. 학습 과정에서는 BERT와 그 위에 쌓은 작은 모듈을 포함한 전체 모델의 출력이 정답 레이블과 최대한 같아지도록 모델 전체를 업데이트합니다.  
이것이 바로 **파인튜닝**입니다.

#### 단어 벡터 활용: 개체명 인식 등  

개체명 인식 같은 과제에서는 마지막 블록의 모든 단어(토큰) 벡터를 활용합니다.  
그림 3-66과 같습니다.

<center><그림 3-66 개체명 인식></center>

<p align="center"><img src="https://i.imgur.com/I0Fdtfe.png">  

<center>출처 : ratsgo's NLPBOOK</center>

그림에서 노란색 박스가 바로 BERT 모델인데요, 이 역시 '빈칸 맞히기'로 프리트레인을 이미 마쳤습니다. 문서 분류 때와 같은 방식으로 입력값을 만들고 BERT의 마지막 레이어까지 계산을 수행합니다. BERT 모델의 마지막 블록(레이어)의 출력은 문장 내 모든 단어(토큰)에 해당하는 벡터들의 시퀀스가 됩니다.

이렇게 뽑은 단어(토큰) 벡터들 위에 작은 모듈을 각각 추가해 그 출력이 각 개체명 범주(**기관명, 인명, 지명** 등)가 될 확률이 되도록 합니다. 학습 과정에서는 BERT와 그 위에 쌓은 각각의 작은 모듈을 포함한 전체 모델의 출력이 정답 레이블과 최대한 같아지도록 모델 전체를 업데이트합니다.

### 문장을 벡터로 변환하기
---
여기서는 프리트레인을 마친 BERT 모델에 문장을 입력해서 이를 벡터로 변환하는 실습을 진행해 보겠습니다.  
이번 실습은 웹 브라우저에서 다음 주소(bit.ly/3BguEuE)에 접속하면 코랩 환경에서 수행할 수 있습니다.

- <**1단계**> **코랩 노트북 초기화** 
  -  이전 실습과 마찬가지로 코랩에서 **'내 드라이브에 복사'**와 **'하드웨어 가속기 사용 안 함'**을 설정합니다. 그리고 다음 코드를 실행해 의존성 있는 패키지를 설치합니다.

In [1]:
#코드 3-11 의존성 패키지 설치
!pip install ratsnlp

     |████████████████████████████████| 42 kB 566 kB/s 
     |████████████████████████████████| 2.8 MB 20.1 MB/s 
     |████████████████████████████████| 806 kB 56.1 MB/s 
     |████████████████████████████████| 57 kB 4.7 MB/s 
     |████████████████████████████████| 636 kB 68.4 MB/s 
     |████████████████████████████████| 829 kB 63.9 MB/s 
     |████████████████████████████████| 397 kB 74.6 MB/s 
     |████████████████████████████████| 134 kB 57.5 MB/s 
     |████████████████████████████████| 67 kB 4.4 MB/s 
     |████████████████████████████████| 3.3 MB 48.2 MB/s 
     |████████████████████████████████| 895 kB 69.9 MB/s 
     |████████████████████████████████| 1.1 MB 54.8 MB/s 
     |████████████████████████████████| 96 kB 3.6 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 144 kB 34.5 MB/s 
     |████████████████████████████████| 271 kB 67.8 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sh

- <**2단계**> **토크나이저 선언**  
    -  BERT 모델의 입력값을 만들려면 토크나이저부터 선언해 두어야 합니다. 다음 코드를 실행하면 **kcbert-base** 모델이 쓰는 토크나이저를 선언합니다.

In [2]:
#코드 3-12 토크나이저 선언
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    "beomi/kcbert-base",
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

- <**3단계**> **모델 초기화**  
    -  다음 단계는 모델을 초기화합니다. 여기서 중요한 것은 사용 대상 BERT 모델이 프리트레인할 때 썼던 토크나이저를 그대로 사용해야 벡터 변환에 문제가 없다는 점입니다. 모델과 토크나이저의 토큰화 방식이 다르면 모델이 엉뚱한 결과를 출력하기 때문이죠. 다음 코드를 실행해 모델을 선언할 때 앞 코드와 똑같은 모델 이름을 적용합니다.

In [3]:
#코드 3-13 모델 선언
from transformers import BertConfig, BertModel
pretrained_model_config = BertConfig.from_pretrained(
    "beomi/kcbert-base"
)
model = BertModel.from_pretrained(
    "beomi/kcbert-base",
    config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


**pretrained_model_config**에는 BERT 모델을 프리트레인할 때 설정했던 내용이 담겨 있습니다. 코랩에서 **pretrained_model_config**를 입력하면 다음과 같은 출력을 확인할 수 있습니다. 블록(레이어)수는 12개, 헤드 수는 12개, 어휘 집합 크기는 3만 개 등의 정보를 확인할 수 있습니다.

In [4]:
pretrained_model_config

BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

코드 3-13의 맨 마지막 줄에서는 이러한 설정에 따라 모델 전체를 초기화한 뒤 미리 학습된 **kcbert-base** 체크포인트를 읽어들이는 역할을 합니다. 체크포인트가 로컬에 저장되어 있지 않다면 웹에서 내려받는 것까지 한 번에 수행합니다.

- <**4단계**> **입력값 만들기**  
    -  다음 코드를 수행하면 BERT 모델의 입력값을 만듭니다.

In [5]:
#코드 3-14 입력값 만들기
sentences = ["안녕하세요", "하이!"]
features = tokenizer(
    sentences,
    max_length=10,
    padding="max_length",
    truncation=True,
)

코드를 실행하고 **features**의 내용을 확인해 보면 다음과 같습니다.

In [7]:
features

{'input_ids': [[2, 19017, 8482, 3, 0, 0, 0, 0, 0, 0], [2, 15830, 5, 3, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

2개의 입력 문장 각각에 대해 워드피스 토큰화를 수행한 뒤 이를 토큰 인덱스로 변환한 결과가 **input_ids**입니다. BERT 모델은 문장 시작에 **CLS**, 끝에 **SEP**라는 스페셜 토큰을 추가하므로 문장 2개 모두 앞뒤에 이들 토큰에 대응하는 인덱스 **2, 3**이 붙었음을 확인할 수 있습니다.

토큰 최대 길이(**max_length**)를 10으로 설정하고, 토큰 길이가 이보다 짧으면 최대 길이에 맞게 패딩(**0**)을 주고(**padding="max_length"**), 길면 자르도록(**truncation=True**) 설정했으므로 **input_ids**의 길이는 두 문장 모두 10인 것을 확인할 수 있습니다.

한편 **attention_mask**는 패딩이 아닌 토큰이 **1**, 패딩인 토큰이 **0**으로 실제 토큰이 자리하는지 아닌지의 정보를 나타냅니다. **token_type_ids**는 세그먼트 정보로 지금처럼 각각이 1개의 문장으로 구성됐을 때는 모두 **0**이 됩니다.

- <**5단계**> **BERT로 단어/문장 수준 벡터 구하기**  
    -  이 책에서는 딥러닝 프레임워크로 파이토치를 쓰고 있는데요, 파이토치 모델의 입력값 자료형은 파이토치에서 제공하는 텐서(tensor)여야 합니다.  
     따라서 코드 3-13에서 만든 파이썬 리스트 형태의 **features**를 텐서로 변환해 줍니다.  

In [10]:
import torch

In [11]:
#코드 3-15 피처를 토치 텐서로 변환
features = {k: torch.tensor(v) for k, v in features.items()}

드디어 BERT 입력값을 다 만들었습니다.  
다음 코드를 실행해 BERT 모델을 실행합니다.

In [12]:
#코드 3-16 임베딩 계산하기
outputs = model(**features)

이 코드의 실행 결과인 **outputs**은 BERT 모델의 여러 출력 결과를 한데 묶은 것입니다. 코랩에서 **outputs.last_hidden_state**를 입력하면 다음과 같은 결과를 볼 수 있습니다.  
이는 문장 2개(**안녕하세요, 하이!**)의 입력 토큰 각각에 해당하는 BERT의 마지막 레이어 출력 벡터들입니다.

In [13]:
outputs.last_hidden_state

tensor([[[-0.6969, -0.8248,  1.7512,  ..., -0.3732,  0.7399,  1.1907],
         [-1.4803, -0.4398,  0.9444,  ..., -0.7405, -0.0211,  1.3064],
         [-1.4299, -0.5033, -0.2069,  ...,  0.1285, -0.2611,  1.6057],
         ...,
         [-1.4406,  0.3431,  1.4043,  ..., -0.0565,  0.8450, -0.2170],
         [-1.3625, -0.2404,  1.1757,  ...,  0.8876, -0.1054,  0.0734],
         [-1.4244,  0.1518,  1.2920,  ...,  0.0245,  0.7572,  0.0080]],

        [[ 0.9371, -1.4749,  1.7351,  ..., -0.3426,  0.8050,  0.4031],
         [ 1.6095, -1.7269,  2.7936,  ...,  0.3100, -0.4787, -1.2491],
         [ 0.4861, -0.4569,  0.5712,  ..., -0.1769,  1.1253, -0.2756],
         ...,
         [ 1.2362, -0.6181,  2.0906,  ...,  1.3677,  0.8132, -0.2742],
         [ 0.5409, -0.9652,  1.6237,  ...,  1.2395,  0.9185,  0.1782],
         [ 1.9001, -0.5859,  3.0156,  ...,  1.4967,  0.1924, -0.4448]]],
       grad_fn=<NativeLayerNormBackward0>)

코랩에서 **outputs.last_hidden_state.shape**를 입력하면 그 셰이프를 확인할 수 있습니다. 바로 [2,10,768]입니다. 문장 2개에 속한 각각의 토큰(시퀀스 길이는 10)이 768차원짜리의 벡터로 변환됐다는 의미입니다.  

cf) 셰이프(shape)란 파이토치 텐서의 크기를 나타내는 개념입니다. 셰이프의 요솟값 수는 텐서의 차원(dimension)을 가리킵니다. 예를 들어 4 × 5 크기의 행렬은 그 셰이프가 [4, 5]인 2차원 텐서가 됩니다. 이러한 행렬을 2개 이어 붙여 3차원 텐서를 만들 수도 있습니다. 그러면 그 셰이프는 [2, 4, 5]가 됩니다. 자연어 처리에서는 보통 [배치 크기, 토큰 수, 토큰 벡터 차원]이라는 3차원 텐서를 사용합니다.

그림 3-67은 **안녕하세요**만 따로 떼어서 그 계산 과정을 나타낸 것입니다. **outputs.last_hidden_state**는 이 그림에서 노란색 점선으로 표기한 벡터들에 대응합니다. 이러한 결과는 개체명 인식 과제처럼 단어별로 수행해야 하는 task에 활용됩니다.  
한편 그림 3-67에서 패딩(0)에 해당하는 토큰들은 셀프 어텐션에서의 상호작용이 제한됩니다. 해당 토큰의 **attention_mask**가 0이기 때문입니다.  

cf) **attention_mask**가 0인 경우 해당 토큰에 대응하는 셀프 어텐션 소프트맥스 확률을 0으로 바꿉니다. 셀프 어텐션 계산에 관한 자세한 내용은 3-3절을 참고하세요!!

코랩에서 **outputs.pooler_output**을 입력해 그 내용을 확인해 보면 다음과 같은 결과를 확인할 수 있습니다.

In [14]:
outputs.pooler_output

tensor([[-0.1594,  0.0547,  0.1101,  ...,  0.2684,  0.1596, -0.9828],
        [-0.9221,  0.2969, -0.0110,  ...,  0.4291,  0.0311, -0.9955]],
       grad_fn=<TanhBackward0>)

이번 출력 결과의 셰이프는 [2, 768]입니다. 문장 2개가 각각 768차원짜리의 벡터로 변환됐다는 의미입니다. 이들은 BERT의 마지막 레이어 **CLS** 벡터에 다음 그림과 같은 처리를 한 결과입니다. 이러한 결과는 문서 분류 과제처럼 문장 전체를 벡터 하나로 변환한 뒤 이 벡터에 어떤 계산을 수행하는 task에 활용됩니다.